In [10]:
from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
docs = fetch_20newsgroups(subset='all')['data']

# find the average length in words of the documents
avg_len = sum([len(doc.split()) for doc in docs]) / len(docs)
avg_len

283*5*4*384/(2**20)



2.07275390625

In [ ]:
embedding = sentence_model.encode("This is a string", show_progress_bar=True)

In [9]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
sentences = ['How big is London', 'How big is Paris?', 'Where is London?', 'Who is Obama?', 'Who is Trump?', 'Who is Cameron?', 'How big is England?', 'How big is France?', 'How big is England?']
embedding = model.encode(sentences,show_progress_bar=True)

embedding.shape

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(9, 384)

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

text = "Your document text. It can contain mulbtiple sentences."
sentences = sent_tokenize(text)
print(sentences)

In [1]:
import pandas as pd
import nltk
from uuid import uuid4
nltk.download('punkt')

def split_and_reconstitute(strings, minimum, maximum, increment, overlap):
    def split_sentences(text):
        """Split the given text into sentences using NLTK."""
        return nltk.sent_tokenize(text)

    def reconstitute(sentences, min_sentences, max_sentences, increment, overlap):
        result = []
        start_index = 0
        doc_id = str(uuid4())

        while start_index < len(sentences):
            end_index = start_index + min_sentences
            chunk_id = str(uuid4())

            while end_index <= len(sentences) and (end_index - start_index) <= max_sentences:
                result.append(' '.join(sentences[start_index:end_index]))
                yield doc_id, chunk_id, start_index, end_index, result[-1]
                end_index += increment

            next_start = end_index - overlap - 1
            if next_start <= start_index:
                break

            start_index = next_start

            if end_index >= len(sentences) and (end_index - start_index) <= min_sentences:
                break

        return result

    data = []
    for string in strings:
        sentences = split_sentences(string)
        for item in reconstitute(sentences, minimum, maximum, increment, overlap):
            data.append(item)

    df = pd.DataFrame(data, columns=['doc_id', 'chunk_id', 'start', 'end', 'string'])
    return df

input_strings = ["This is a test. Testing sentence splitting. Another sentence here. This is another string. It has two sentences. This is a third string. It has three sentences.",
                 "This is a test. Testing sentence splitting. Another sentence here. This is another string. It has two sentences. This is a third string. It has three sentences."]

from sklearn.datasets import fetch_20newsgroups
from sentence_transformers import SentenceTransformer
from data_processing.example import create_example_input_docs


# Restrict to 2500 documents for testing
docs = create_example_input_docs(2500)
model = SentenceTransformer('all-MiniLM-L6-v2')

result = split_and_reconstitute(docs, 1, 3, 1, 1)
result.shape

#350 seems to be a sweet spot
result['embeddings'] = model.encode(result['string'].tolist(), show_progress_bar=True,batch_size=250, device="cuda").tolist()
result

[nltk_data] Downloading package punkt to /home/guy/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
2024-01-12 16:28:43,162 - INFO - data_processing.example - Creating example data
2024-01-12 16:28:44,835 - INFO - data_processing.example - Created 2500 example records


Batches:   0%|          | 0/159 [00:00<?, ?it/s]

,doc_id,chunk_id,start,end,string,embeddings
0,70b8a19c-6b1d-4980-8350-c5c1b0862672,ac7c5449-29ac-494e-9f97-9969579616bd,0,1,I got the offer to upgrade this weekend.,"[-0.05333279073238373, -0.005516815930604935, ..."
1,70b8a19c-6b1d-4980-8350-c5c1b0862672,ac7c5449-29ac-494e-9f97-9969579616bd,0,2,I got the offer to upgrade this weekend. It's ...,"[-0.05075643211603165, 0.015621621161699295, 0..."
2,70b8a19c-6b1d-4980-8350-c5c1b0862672,ac7c5449-29ac-494e-9f97-9969579616bd,0,3,I got the offer to upgrade this weekend. It's ...,"[-0.033895041793584824, -0.01855466328561306, ..."
3,70b8a19c-6b1d-4980-8350-c5c1b0862672,f127fc2a-2343-432d-8fd7-b9d1c55bc6e1,2,3,The S+H seem way too steep for just a couple o...,"[-0.0467366948723793, -0.02633601427078247, -0..."
4,70b8a19c-6b1d-4980-8350-c5c1b0862672,f127fc2a-2343-432d-8fd7-b9d1c55bc6e1,2,4,The S+H seem way too steep for just a couple o...,"[-0.03360332176089287, -0.015437657944858074, ..."
...,...,...,...,...,...,...
39525,d5e58306-3b8a-4652-90b7-47bde010fa21,6bda2138-9fc3-4587-97ad-d03b2f91f6a6,3,4,Feng,"[-0.07738784700632095, 0.06844016909599304, 0...."
39526,34c38175-aeca-458d-9582-4579089dda53,cedb6484-f1be-412a-ab56-22f5ef0bb15f,0,1,"Hello out there,\nIf your familiar with the CO...","[-0.03322175145149231, -0.050955597311258316, ..."
39527,34c38175-aeca-458d-9582-4579089dda53,cedb6484-f1be-412a-ab56-22f5ef0bb15f,0,2,"Hello out there,\nIf your familiar with the CO...","[0.024918068200349808, -0.03647035360336304, 0..."
39528,34c38175-aeca-458d-9582-4579089dda53,cedb6484-f1be-412a-ab56-22f5ef0bb15f,0,3,"Hello out there,\nIf your familiar with the CO...","[-0.00042022584239020944, -0.03625414147973060..."
